In [2]:
N = -117299665605343495500066013555546076891571528636736883265983243281045565874069282036132569271343532425435403925990694272204217691971976685920273893973797616802516331406709922157786766589075886459162920695874603236839806916925542657466542953678792969287219257233403203242858179791740250326198622797423733569670 + 617172569155876114160249979318183957086418478036314203819815011219450427773053947820677575617572314219592171759604357329173777288097332855501264419608220917546700717670558690359302077360008042395300149918398522094125315589513372914540059665197629643888216132356902179279651187843326175381385350379751159740993*I
a = 1671911043329305519973004484847472037065973037107329742284724545409541682312778072234 * 10^70 + + 193097758392744599866999513352336709963617764800771451559221624428090414152709219472155 * 10^68 * I

In [3]:
def coefficient_poly(g, m):
    assert(g.degree() <= m)
    unpadded_result = g.coefficients(sparse=False)
    result = unpadded_result + [0] * (m - len(unpadded_result))
    return vector(result)

# get block matrix canonical embedding of Gaussian integer z, scaled with lmbds 
def get_block_matrix(z, s, limits):
    return matrix(ZZ, [[limits[0]^s * z.real(), limits[1]^s * z.imag()], [-1 * limits[0]^s * z.imag(), limits[1]^s * z.real()]])

def complex_coppersmith(f, N, limits, k=3, t=3):
    x = f.parent().gen(0)
    d = f.degree()
    m = d * k + t
    blocks = []
    for i in range(k):
        for j in range(d):
            poly = x^j * f^i * N^(k-i)
            poly_coeffs = coefficient_poly(poly, m)
            block_matrix_row = [get_block_matrix(z, s, limits) for (s, z) in enumerate(poly_coeffs)]
            blocks.append(block_matrix_row)
    for j in range(t):
        poly = x^j * f^k
        poly_coeffs = coefficient_poly(poly, m)
        block_matrix_row = [get_block_matrix(z, s, limits) for (s, z) in enumerate(poly_coeffs)]
        blocks.append(block_matrix_row)
    
    M = block_matrix(blocks)
    
    M_reduced = M.LLL(); v = M_reduced[0]

    Q = 0
    for (s, i) in enumerate(list(range(0, len(v), 2))):
        z = v[i] / (limits[0]^s) + v[i+1] / (limits[1]^s) * I
        Q += z * x^s
    return Q

In [4]:
R.<x> = PolynomialRing(I.parent(), "x")
f = x + a; f
Q = complex_coppersmith(f, N, [10^70, 10^68], k=10, t=10)

In [5]:
Q.roots()

[(38885530450742215892390797656090317247451645952898238958411902273365*I + 7969971216531422897795829216783415368573272798190263112723158574187978,
  1)]

In [6]:
p = a + Q.roots()[0][0]
p

19309775839274459986699951335233670996361776480077145155922162442809041415270921947215538885530450742215892390797656090317247451645952898238958411902273365*I + 16719110433293055199730044848474720370659730371073297422847245454095416823127780722347969971216531422897795829216783415368573272798190263112723158574187978

In [7]:
p.norm()

652396096643676367544566156124913216163410921734827378316264717178043201248098216744333182034733046461246836969083455954245444287495898970654732184830855419214640885149170735361071243374488441853305632933085361898289750475979418539227694088799586535639304205925186104659033478286855260238037651424137268151709

In [10]:
N.norm() / p.norm()

604941068317189652627858851523167292428948871043205807414277515111804456754639695955081185290003101108400607518848813115420632093104992473355039185637288241681868004615473802877209913302613123923904488351942249382653677543786493647849046273946794244961213693227449138212955169271151864267238394436697214152361